In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# Define transformaciones para preprocesar las imágenes
transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Redimensionar las imágenes a un tamaño específico
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=np.random.choice(range(0, 360))), 
    transforms.ToTensor(),          # Convertir las imágenes a tensores
    transforms.Normalize(0.5, 0.5)  # Normalizar los valores de píxeles
])


In [3]:
# Directorio que contiene los datos de entrenamiento y validación
train_dir = '/home/carlos/Documentos/proyectos/breast_cancer/Datasets/prueba_class_masks/train/'
val_dir = '/home/carlos/Documentos/proyectos/breast_cancer/Datasets/prueba_class_masks/val/'

# Crear datasets para entrenamiento y validación
train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data = datasets.ImageFolder(val_dir, transform=transform)

# Crear dataloaders para cargar los datos
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)
val_loader = DataLoader(val_data, batch_size=16)

In [19]:

# Definir una red neuronal convolucional simple
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, 3, padding=1)
        self.conv2 = nn.Conv2d(4, 8, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.drop = nn.Dropout(.4)
        self.flat = nn.Flatten()
        self.fc1 = nn.Linear(8 * 128 * 128, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 16)
        self.fcout = nn.Linear(16, 3) 

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
#         x = x.view(-1, 16 * 16 * 16)
        x = self.flat(x)
        x = self.drop(torch.relu(self.fc1(x)))
        x = self.drop(torch.relu(self.fc2(x)))
        x = self.drop(torch.relu(self.fc3(x)))
        x = self.fcout(x)
        return x

# Crear una instancia del modelo
model = SimpleCNN()

# Definir la función de pérdida y el optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [20]:
# Training the model
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    accuracy = (correct / total) * 100
    train_loss = running_loss / len(train_loader)


    # Validation phase
    model.eval()  # Set the model to evaluation mode
    runing_val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data[0].to(device), data[1].to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            runing_val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
        
        val_accuracy = (val_correct / val_total) * 100
        val_loss = runing_val_loss / len(val_loader)

    print(f'Epoch {epoch + 1} Train Loss: {train_loss:.4f} Validation Loss: {val_loss:.4f}, '
          f'Train accuracy: {accuracy:.2f}%  Val accuracy: {val_accuracy:.2f}%')

print("Training completed")

Epoch 1 Train Loss: 1.3497 Validation Loss: 1.0841, Train accuracy: 36.11%  Val accuracy: 37.78%
Epoch 2 Train Loss: 1.0269 Validation Loss: 1.0569, Train accuracy: 43.33%  Val accuracy: 46.67%
Epoch 3 Train Loss: 1.0476 Validation Loss: 1.0243, Train accuracy: 40.56%  Val accuracy: 56.67%
Epoch 4 Train Loss: 1.0070 Validation Loss: 1.0364, Train accuracy: 42.22%  Val accuracy: 56.67%
Epoch 5 Train Loss: 0.9887 Validation Loss: 1.0139, Train accuracy: 51.11%  Val accuracy: 61.11%
Epoch 6 Train Loss: 0.9973 Validation Loss: 0.9973, Train accuracy: 49.72%  Val accuracy: 62.22%
Epoch 7 Train Loss: 0.9787 Validation Loss: 1.0042, Train accuracy: 52.50%  Val accuracy: 60.00%
Epoch 8 Train Loss: 0.9497 Validation Loss: 1.0114, Train accuracy: 56.94%  Val accuracy: 62.22%
Epoch 9 Train Loss: 0.9813 Validation Loss: 1.0162, Train accuracy: 54.17%  Val accuracy: 46.67%
Epoch 10 Train Loss: 0.9731 Validation Loss: 1.0035, Train accuracy: 53.89%  Val accuracy: 60.00%
Epoch 11 Train Loss: 0.9616 V

Epoch 85 Train Loss: 0.7619 Validation Loss: 0.8905, Train accuracy: 64.44%  Val accuracy: 67.78%
Epoch 86 Train Loss: 0.7944 Validation Loss: 0.8973, Train accuracy: 65.28%  Val accuracy: 65.56%
Epoch 87 Train Loss: 0.7542 Validation Loss: 0.8412, Train accuracy: 66.39%  Val accuracy: 64.44%
Epoch 88 Train Loss: 0.7577 Validation Loss: 0.8695, Train accuracy: 65.83%  Val accuracy: 66.67%
Epoch 89 Train Loss: 0.7574 Validation Loss: 0.8625, Train accuracy: 67.78%  Val accuracy: 63.33%
Epoch 90 Train Loss: 0.7363 Validation Loss: 0.8586, Train accuracy: 67.50%  Val accuracy: 66.67%
Epoch 91 Train Loss: 0.7298 Validation Loss: 0.8546, Train accuracy: 67.50%  Val accuracy: 63.33%
Epoch 92 Train Loss: 0.7386 Validation Loss: 0.8674, Train accuracy: 67.22%  Val accuracy: 65.56%
Epoch 93 Train Loss: 0.7204 Validation Loss: 0.8637, Train accuracy: 69.17%  Val accuracy: 64.44%
Epoch 94 Train Loss: 0.7223 Validation Loss: 0.8860, Train accuracy: 67.22%  Val accuracy: 63.33%
Epoch 95 Train Loss:

In [ ]:
# Guardar el modelo entrenado
torch.save(model.state_dict(), 'modelo_entrenado.pth')